# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 5. Klasifikasi Decision Tree


### 1) Import Data

Praktikum kali ini menggunakan dataset [Car Evaluation Dataset](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation) dari UCI Machine Learning Repository. Dataset ini telah digunakan pada praktikum sebelumnya. Detail fitur dapat Anda pelajari pada link yang tersedia. 

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [98]:
## Sudah di-download

Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [99]:
import pandas as pd

df = pd.read_csv('car_evaluation.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [100]:
df.head()

,buying,maint,lug_boot,safety,class
0,vhigh,vhigh,small,low,unacc
1,vhigh,vhigh,small,med,unacc
2,vhigh,vhigh,small,high,unacc
3,vhigh,vhigh,med,low,unacc
4,vhigh,vhigh,med,med,unacc


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**. Agar pengacakan data dilakukan secara konstan, parameter **random_state** diisi dengan nilai integer tertentu, pada praktikum ini diset 101. Kemudian, nilai indeks pada data latih dan data uji diatur ulang agar berurutan nilainya


In [101]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2, random_state=101)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 208 data, dan **data_uji** terdiri dari 52 data

In [102]:
print(df_train.shape[0])
print(df_test.shape[0])

1382
346


## 3) Menghitung Gini

Nilai Gini merupakan salah satu kriteria penentu variabel apa yang akan digunakan untuk membentuk cabang pada decision tree. Variabel dengan nilai Gini terbesar akan digunakan sebagai pembentukan cabang

Buatlah fungsi bernama **hitung_gini** yang berfungsi menghitung nilai Gini dari suatu nilai pada sebuah variabel

In [103]:
import numpy as np
def count_gini(class_column):
  element, count = np.unique(class_column, return_counts=True)
  gini_val = 1 - np.sum([count[i]]/np.sum(count)**2 for i in range(len(element)))
  return gini_val

Buatlah fungsi bernama **gini_split** yang digunakan untuk menghitung nilai Gini keseluruhan dari sebuah variabel.

In [104]:
def gini_split(df, split_name_feature, class_name_feature):
  val, count = np.unique(df[split_name_feature], return_counts=True)
  gini_split_val = np.sum([(count[i]/np.sum(count))*count_gini(df.where(df[split_name_feature]==val[i]).dropna()[class_name_feature]) for i in range(len(val))])
  return gini_split_val

Ujilah fungsi **gini_split** menggunakan data_latih pada variabel **buying** dan variabel kelas bernama **class**.

In [105]:
print(gini_split(df_train, 'buying', 'class'))

0.9971056439942112


C:\Users\indra\AppData\Local\Temp\ipykernel_5628\1677230531.py:4: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  gini_val = 1 - np.sum([count[i]]/np.sum(count)**2 for i in range(len(element)))
C:\Users\indra\AppData\Local\Temp\ipykernel_5628\1677230531.py:4: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  gini_val = 1 - np.sum([count[i]]/np.sum(count)**2 for i in range(len(element)))
C:\Users\indra\AppData\Local\Temp\ipykernel_5628\1677230531.py:4: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  gini_val = 1 - np.sum([count[i]]/np.sum(count)**2 for i in range(len(element)))
C:\Users\indra\AppD

## 4) Pembentukan pohon

Pembentukan pohon dilakukan secara rekursif. Seperti metode rekursif pada umumnya, perlu ditentukan kondisi berhenti terlebih dahulu. Kondisi berhenti pada pembentukan pohon adalah:


1.   Jika hanya ada satu kelas pada data, kembalikan kelas tersebut
2.   Jika fitur data  = 0 (tidak ada fitur yang tersisa), kembalikan kelas dari parent
3. Jika data kosong (tidak ada data), kembalikan kelas dengan frekuensi terbanyak

Selain kondisi berhenti tersebut, dilakukan pembentukan pohon secara rekursif menggunakan fungsi **buat_tree**.



In [106]:
def create_tree(df, begin_df, list_feature, class_name_feature, parent_class_node=None):
  if len(np.unique(df[class_name_feature])) <= 1:
    return np.unique(df[class_name_feature])[0]
  elif len(df) == 0:
    return np.unique(begin_df[class_name_feature])[np.argmax(np.unique(begin_df[class_name_feature], return_counts=True)[1])]
  elif len(list_feature) == 0:
    return parent_class_node
  else:
    parent_class_node = np.unique(df[class_name_feature])[np.argmax(np.unique(df[class_name_feature], return_counts=True)[1])]
    split_val = [gini_split(df, feature, class_name_feature) for feature in list_feature]
    best_index_feature = np.argmin(split_val)
    best_feature = list_feature[best_index_feature]
    tree = {best_feature:{}}
    list_feature = [i for i in list_feature if i != best_feature]
    for value in np.unique(df[best_feature]):
      sub_data = df.where(df[best_feature] == value).dropna()
      subtree = create_tree(sub_data, begin_df, list_feature, class_name_feature, parent_class_node)
      tree[best_feature][value] = subtree
      
    return (tree)

Buatlah tree menggunakan data latih yang tersedia

In [107]:
tree = create_tree(df_train, df_train, df_train.columns[:-1], 'class')

C:\Users\indra\AppData\Local\Temp\ipykernel_5628\1677230531.py:4: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  gini_val = 1 - np.sum([count[i]]/np.sum(count)**2 for i in range(len(element)))
C:\Users\indra\AppData\Local\Temp\ipykernel_5628\1677230531.py:4: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  gini_val = 1 - np.sum([count[i]]/np.sum(count)**2 for i in range(len(element)))
C:\Users\indra\AppData\Local\Temp\ipykernel_5628\1677230531.py:4: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  gini_val = 1 - np.sum([count[i]]/np.sum(count)**2 for i in range(len(element)))
C:\Users\indra\AppD

C:\Users\indra\AppData\Local\Temp\ipykernel_5628\1677230531.py:4: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  gini_val = 1 - np.sum([count[i]]/np.sum(count)**2 for i in range(len(element)))
C:\Users\indra\AppData\Local\Temp\ipykernel_5628\1677230531.py:4: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  gini_val = 1 - np.sum([count[i]]/np.sum(count)**2 for i in range(len(element)))
C:\Users\indra\AppData\Local\Temp\ipykernel_5628\1677230531.py:4: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  gini_val = 1 - np.sum([count[i]]/np.sum(count)**2 for i in range(len(element)))
C:\Users\indra\AppD

Tampilkan tree yang terbentuk. Gunakan library **pprint** untuk menampilkan dictionary secara teratur.

In [108]:
from pprint import pprint
pprint(tree)

{'buying': {'high': {'maint': {'high': {'safety': {'high': {'lug_boot': {'big': 'acc',
                                                                         'med': 'acc',
                                                                         'small': 'acc'}},
                                                   'low': 'unacc',
                                                   'med': {'lug_boot': {'big': 'unacc',
                                                                        'med': 'unacc',
                                                                        'small': 'unacc'}}}},
                               'low': {'lug_boot': {'big': {'safety': {'high': 'unacc',
                                                                       'low': 'unacc',
                                                                       'med': 'unacc'}},
                                                    'med': {'safety': {'high': 'unacc',
                                              

## 5) Proses prediksi

Proses prediksi kelas pada data uji dilakukan dengan melakukan *tree traversal* sampai menemui leaf.

In [109]:
def predict(df_test, tree):
  for key in list(df_test.keys()):
    if key in list(tree.keys()):
      try:
        result = tree[key][df_test[key]]
      except:
        return 1
      result = tree[key][df_test[key]]
      if isinstance(result, dict):
        return predict(df_test, result)
      else:
        return result

## 6) Proses Pengujian
Lakukan pengujian menggunakan data uji. Kelas pada data uji perlu dihapus dan data uji perlu diubah menjadi dictionary

In [110]:
df_test_dict = df_test.iloc[:,:-1].to_dict(orient='records')

Lakukan pengujian terhadap keseluruhan data uji menggunakan looping.

In [111]:
prediction_result_total = []
for i in range(len(df_test_dict)):
  prediction_result = predict(df_test_dict[i], tree)
  prediction_result_total.append(prediction_result)

Bandingkan hasil prediksi dengan label sebenarnya. Hitunglah banyaknya data uji yang memiliki kelas prediksi sama dengan kelas sebenarnya

In [112]:
print("Total prediksi benar: ", sum(prediction_result_total == df_test['class']))

Total prediksi benar:  257


## TUGAS
Pada tugas kali ini Anda diminta memodifikasi metode pembentukan tree yang telah Anda agar metode tersebut menggunakan information gain sebagai dasar percabangan. Lengkapilah kerangka source code di bawah ini

Lengkapi fungsi hitung_entropy

In [113]:
def hitung_entropy(kolom_kelas):
  element, count = np.unique(kolom_kelas, return_counts=True)
  entropy = -np.sum([(count[i]/np.sum(count))*np.log2(count[i]/np.sum(count)) for i in range(len(element))])
  return entropy

Lengkapi fungsi information_gain

In [114]:
def information_gain(data, nama_fitur_split, nama_fitur_kelas):
  total_entropy = hitung_entropy(data[nama_fitur_kelas])
  val, count = np.unique(data[nama_fitur_split], return_counts=True)
  split_entropy = np.sum([(count[i]/np.sum(count))*hitung_entropy(data.where(data[nama_fitur_split]==val[i]).dropna()[nama_fitur_kelas]) for i in range(len(val))])
  information_gain = total_entropy - split_entropy
  return information_gain

Lengkapi fungsi **buat_tree_ig**. Isinya sama persis dengan fungsi **buat_tree**, hanya saja penghitungan **gini_split** diganti dengan **information_gain**. Selain itu, percabangan dilakukan dengan menggunakan nilai **information_gain** **terbesar**

In [115]:
def buat_tree_ig(data,data_awal, daftar_fitur, nama_fitur_kelas,kelas_parent_node=None):
  if len(np.unique(data[nama_fitur_kelas])) <= 1:
    return np.unique(data[nama_fitur_kelas])[0]
  elif len(data)==0:
    return np.unique(data_awal[nama_fitur_kelas])[np.argmax(np.unique(data_awal[nama_fitur_kelas], return_counts=True)[1])]
  elif len(daftar_fitur)==0:
    return kelas_parent_node
  else:
    kelas_parent_node = np.unique(data[nama_fitur_kelas])[np.argmax(np.unique(data[nama_fitur_kelas], return_counts=True)[1])]
    ig = [information_gain(data, fitur, nama_fitur_kelas) for fitur in daftar_fitur]
    split = np.argmax(ig)
    best_fitur = daftar_fitur[split]
    tree = {best_fitur:{}}
    daftar_fitur = [i for i in daftar_fitur if i != best_fitur]
    for value in np.unique(data[best_fitur]):
      sub_data = data.where(data[best_fitur] == value).dropna()
      subtree = buat_tree_ig(sub_data, data_awal, daftar_fitur, nama_fitur_kelas, kelas_parent_node)
      tree[best_fitur][value] = subtree
  return(tree)

Lakukan pembentukan tree menggunakan fungsi **buat_tree_ig**

In [116]:
tree_ig = buat_tree_ig(df_test,df_test,df_test.columns[:-1],'class')

Tampilkan tree yang terbentuk

In [117]:
pprint(tree_ig)

{'safety': {'high': {'buying': {'high': {'maint': {'high': {'lug_boot': {'big': 'acc',
                                                                         'med': 'acc',
                                                                         'small': 'acc'}},
                                                   'low': {'lug_boot': {'big': 'acc',
                                                                        'med': 'acc',
                                                                        'small': 'acc'}},
                                                   'med': {'lug_boot': {'big': 'acc',
                                                                        'med': 'unacc',
                                                                        'small': 'unacc'}},
                                                   'vhigh': 'unacc'}},
                                'low': {'maint': {'high': {'lug_boot': {'med': 'acc',
                                                  

Lakukan pengujian menggunakan tree yang terbentuk

In [118]:
hasil_prediksi_total_ig = []
for i in range(len(df_test_dict)):
  hasil_prediksi = predict(df_test_dict[i],tree_ig)
  hasil_prediksi_total_ig.append(hasil_prediksi)
print("Total prediksi benar: ",sum(hasil_prediksi_total_ig==df_test['class']))

Total prediksi benar:  292


# PERTANYAAN

Jawablah pertanyaan di bawah ini



1.   Amati tree yang dihasilkan dengan kriteria percabangan GINI dan Information Gain. Apa perbedaan tree yang dihasilkan dari kedua metode tersebut?
2.   Apakah penggunaan Information Gain dapat meningkatkan akurasi prediksi?



Tulis jawaban Anda di cell ini


1. Algoritma pohon keputusan seperti CART (Classification and Regression Trees) dapat menggunakan dua metode utama untuk membangun pohon: Gini Impurity dan Information Gain (biasanya diukur dengan Entropy). Perbedaan utama antara keduanya terletak pada cara mereka mengukur ketidakmurnian atau keberagaman dalam data.

   - **Gini Impurity**: Mengukur seberapa sering elemen dari sebuah set data akan salah diklasifikasikan jika mereka dipilih secara acak. Ini berfokus pada perbedaan di antara kelas target.
   
   - **Information Gain**: Mengukur seberapa banyak informasi yang diberikan oleh fitur tertentu dalam mengurangi ketidakpastian tentang kelas target. Ini berfokus pada pengurangan ketidakpastian atau entropy dalam kelas target setelah membagi data berdasarkan fitur tersebut.

   Dengan demikian, pohon keputusan yang dihasilkan dari kedua metode tersebut mungkin berbeda dalam struktur dan cara fitur-fitur dipilih untuk membuat keputusan cabang.

2. Penggunaan Information Gain biasanya dapat meningkatkan akurasi prediksi karena algoritma akan cenderung memilih fitur-fitur yang memberikan informasi yang paling berguna untuk membagi data menjadi kelas yang berbeda. Namun, ini tidak selalu terjadi dalam setiap kasus. Terkadang, Gini Impurity dapat bekerja lebih baik tergantung pada data dan kompleksitas masalahnya. Oleh karena itu, penting untuk mencoba kedua metode dan melakukan validasi silang untuk menentukan mana yang memberikan kinerja terbaik pada dataset tertentu.
